In [ ]:
import os
import pandas as pd
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()

from keras.layers import Input, Dense
from keras.models import Model


algorithm = 'RF_corr_ratio_ap'

# RF/XGB
# corr/auto
# threshold/ratio
# roc/ap
# paragrid

patents_path='./patents SMILES/'
topology_path='./topology/'
descriptor_path='./descriptor/'
basic_path='./basic/'
result_path='./result/'

df = pd.read_excel('drugbank_extract_filtered.xlsx')
drug_name_list = df['Generic Name'].tolist()
compound_num_list = df['compounds number'].tolist()

drug_list = []
for i in range(0,len(df)):
  one = str(compound_num_list[i]) + '_' + drug_name_list[i] + '.xlsx'
  drug_list.append(one)

drug_list = sorted(drug_list, key=lambda x: int(x.split('_')[0]))
print(len(drug_list))

In [ ]:
def nested_cv(X,y):
    X1 = X.values

    if 'RF' in algorithm:
        model = RandomForestClassifier(random_state=1)
        param_grid = {
            'n_estimators':[100,200,300],
            'class_weight':[{1:50,0:1}],
            'max_depth':[3,6,9]
        }

        param_grid = {
            'n_estimators':[300],
            'class_weight':[{1:50,0:1}],
            'max_depth':[6]
        }

    if 'XGB' in algorithm:
        model = xgb.XGBClassifier(seed=1,tree_method = "hist")
        param_grid = {
        'n_estimators':[100,200,300],
        'scale_pos_weight':[50],
        'max_depth':[3,6,9]
        }

        param_grid = {
        'n_estimators':[100],
        'scale_pos_weight':[50],
        'max_depth':[3]
        }

    # Set up outer cross-validation loop
    outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

    metrics_scores = []
    best_params = []
    cm_list = []

    # Execute outer splits
    for train_index, test_index in outer_cv.split(X1, y):
        X_train, X_test = X1[train_index, :], X1[test_index, :]
        y_train, y_test = y[train_index], y[test_index]

        # 数据预处理
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        if 'RF' in algorithm:
            X_train, X_test = np.nan_to_num(X_train, nan=0), np.nan_to_num(X_test, nan=0)

        # Set up inner cross-validation loop
        inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=1)

        # Create GridSearchCV object
        if 'roc' in algorithm:
            clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring='roc_auc', error_score='raise',n_jobs=9)
        if 'ap' in algorithm:
            clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring='average_precision', error_score='raise',n_jobs=9)

        # Fit GridSearchCV on the training set
        clf.fit(X_train, y_train)

        # Predict probabilities on the test set
        y_pred_proba = clf.predict_proba(X_test)

        # Calculate the confusion matrix
        y_pred = clf.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        cm_list.append(cm)
        
        # Calculate scores for the fold
        if 'roc' in algorithm:
            metrics_score = roc_auc_score(y_test, y_pred_proba[:, 1])
        if 'ap' in algorithm:
            metrics_score = average_precision_score(y_test, y_pred_proba[:, 1])
        metrics_scores.append(metrics_score)
        best_params.append(clf.best_params_)

        # 排序参数重要性
        best_model = clf.best_estimator_
        feature_importance = best_model.feature_importances_
        sorted_idx = feature_importance.argsort()
        top_sorted_idx = sorted_idx[-10:]

        # 绘制重要参数图表
        plt.figure(figsize=(10, 6))
        plt.barh(range(len(top_sorted_idx)), feature_importance[top_sorted_idx], align='center')
        plt.yticks(range(len(top_sorted_idx)), [X.columns[i] for i in top_sorted_idx])
        plt.xlabel('Feature Importance')
        plttitle =  algorithm + ' Top 10 features ' + str(cutoff) + ', ' +str(len(metrics_scores))
        plt.title(plttitle)
        plt.show()
        

    # Calculate average scores across all folds
    average_metrics_score = np.mean(metrics_scores)

    # Calculate std across all folds
    std_metrics_score = np.std(metrics_scores)

    # Determine the best parameters based on the highest average scores
    best_params_overall = best_params[np.argmax(metrics_scores)]

    best_cm = cm_list[np.argmax(metrics_scores)]
    best_accuracy = best_cm[1][1]/(best_cm[1][0]+best_cm[1][1])
    print(best_cm)

    return best_params_overall, average_metrics_score, std_metrics_score, best_cm, best_accuracy

In [ ]:
if 'corr' in algorithm:
    # reduce_dimention = pd.DataFrame()
    # for drug in drug_list:
    #     one_descriptor=pd.read_excel(descriptor_path+'descriptor_'+drug)
    #     reduce_dimention = pd.concat([reduce_dimention, one_descriptor],axis=0,ignore_index=True)

    # # 降维descriptor，选择与其他特征平均相关性较低的前64个特征
    # correlation_matrix = reduce_dimention.iloc[:, 1:].corr()
    # avg_correlation = correlation_matrix.mean()
    # top_features = avg_correlation.nsmallest(64).index

    # # 创建DataFrame保存特征和对应的平均相关性
    # df = pd.DataFrame({'Feature': top_features, 'Average Correlation': [avg_correlation[feature] for feature in top_features]})

    # # 保存DataFrame到本地CSV文件
    # df.to_csv('top_features_correlation.csv', index=False)

    # print("Table saved to 'top_features_correlation.csv'.")

    top_features = ['HallKierAlpha', 'BCUT2D_CHGLO', 'BCUT2D_LOGPLOW', 'VSA_EState5',
    'BalabanJ', 'FpDensityMorgan1', 'MinAbsEStateIndex', 'MinEStateIndex',
    'qed', 'BCUT2D_MRLOW', 'BCUT2D_MWLOW', 'MinPartialCharge',
    'VSA_EState7', 'FpDensityMorgan2', 'fr_aldehyde', 'fr_isocyan', 'Ipc',
    'Kappa3', 'fr_nitroso', 'NumRadicalElectrons', 'fr_dihydropyridine',
    'fr_hdrzine', 'fr_thiocyan', 'fr_C_S', 'FractionCSP3', 'fr_azide',
    'fr_diazo', 'fr_azo', 'fr_barbitur', 'fr_Ar_COO', 'fr_term_acetylene',
    'fr_SH', 'fr_quatN', 'fr_isothiocyan', 'fr_benzodiazepine',
    'fr_epoxide', 'fr_oxazole', 'fr_phos_acid', 'fr_amidine', 'fr_hdrzone',
    'fr_furan', 'fr_oxime', 'fr_imide', 'fr_HOCCN', 'fr_N_O', 'fr_lactam',
    'fr_nitro', 'MaxAbsPartialCharge', 'fr_nitro_arom_nonortho',
    'fr_thiophene', 'fr_morpholine', 'fr_nitrile', 'fr_sulfone',
    'fr_nitro_arom', 'fr_tetrazole', 'fr_alkyl_halide', 'fr_phos_ester',
    'BCUT2D_MWHI', 'MinAbsPartialCharge', 'fr_unbrch_alkane', 'fr_ArN',
    'SlogP_VSA7', 'fr_allylic_oxid', 'fr_sulfonamd']

In [ ]:
if 'auto' in algorithm:

    reduce_dimention = pd.DataFrame()
    for drug in drug_list:
        one_descriptor=pd.read_excel(descriptor_path+'descriptor_'+drug)
        reduce_dimention = pd.concat([reduce_dimention, one_descriptor],axis=0,ignore_index=True)

    reduce_dimention=reduce_dimention.iloc[:, 1:].values


    reduce_dimention = np.nan_to_num(reduce_dimention, nan=0)
    reduce_dimention = scaler.fit_transform(reduce_dimention)
    # 构建autoencoder模型
    input_img = Input(shape=(reduce_dimention.shape[1],))
    encoded = Dense(128, activation='relu')(input_img)
    encoded = Dense(64, activation='relu')(encoded)

    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dense(reduce_dimention.shape[1], activation='sigmoid')(decoded)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    # 训练autoencoder
    autoencoder.fit(reduce_dimention, reduce_dimention, epochs=500, batch_size=256, shuffle=True)

    # 提取autoencoder的中间层作为特征
    encoder = Model(input_img, encoded)

    # 生成特征
    reduce_dimention_encoded = encoder.predict(reduce_dimention)

    # 将 NumPy 数组转换为 DataFrame，并使用自动生成的列名
    np_df = pd.DataFrame(reduce_dimention_encoded, columns=[f'feature_{i}' for i in range(reduce_dimention_encoded.shape[1])])


In [ ]:
def find_closest_thres(lst, target):
    closest_index = None
    min_diff = float('inf')  # 初始化最小差值为正无穷
    
    for i, num in enumerate(lst):
        diff = abs(num - target)  # 计算当前元素与目标值的差值
        
        if diff < min_diff:  # 如果当前差值更小，则更新最小差值和对应的索引
            min_diff = diff
            closest_index = i
    
    return (closest_index + 1)/10

# 选定一个ratio，然后每个药物选择最接近的threshold，做成字典，后续模型参数输入


if 'ratio' in algorithm:
    ratio_df = pd.read_excel('residue_ratio.xlsx')
    ratios = [round(i * 0.1, 1) for i in range(1, 10)]

    ratio_thres_df = pd.DataFrame()
    for drug in drug_list:
        drugname = drug.split('.')[0].split('_')[1]
        dict1={}
        dict1['drug_name'] = drug.split('.')[0].split('_')[1]
        ratio_list = ratio_df[ratio_df['drug_name'] == drugname].values.tolist()[0][1:]
        for ratio in ratios:
            
            pair_threshold = find_closest_thres(ratio_list, ratio)
            dict1[ratio] = pair_threshold
        new_row =  pd.DataFrame(dict1, index=[0])
        ratio_thres_df = pd.concat([ratio_thres_df, new_row], ignore_index=True)

    print(ratio_thres_df)
        

In [ ]:
cutoffs = [round(i * 0.1, 1) for i in range(1, 10)]
# 每一个ratio都把全部药物(可能不同threshold)加载进来，然后丢进去cv训练出一个结果

result_all = pd.DataFrame()
for cutoff in tqdm(cutoffs):

    params = []
    aver_score = []
    std_score = []
    cm_score = []
    accuracy_score = []
    
    drug_all = pd.DataFrame()
    for drug in drug_list:
        basic=pd.read_excel(basic_path+'basic_'+drug)

        if 'ratio' in algorithm:
            cutoff_ = ratio_thres_df.loc[ratio_thres_df['drug_name'] == drug.split('.')[0].split('_')[1], cutoff].values[0]
            topology=pd.read_excel(topology_path+drug.split('.')[0].split('_')[1]+'/tani_ECFP6_'+str(cutoff_)+'_'+drug)
        if 'threshold' in algorithm:
            topology=pd.read_excel(topology_path+drug.split('.')[0].split('_')[1]+'/tani_ECFP6_'+str(cutoff)+'_'+drug)

        if 'corr' in algorithm:
            descriptor=pd.read_excel(descriptor_path+'descriptor_'+drug)
            drug_info = pd.merge(pd.merge(basic, topology, on='c_smiles'), descriptor, on='c_smiles')
        if 'auto' in algorithm:
            drug_info = pd.merge(basic, topology, on='c_smiles')
            
        drug_all = pd.concat([drug_all, drug_info],axis=0,ignore_index=True)
    
    if 'corr' in algorithm:
        drug_all = pd.concat([drug_all.iloc[:, :14], drug_all[top_features]],axis=1)
    if 'auto' in algorithm:
        drug_all = pd.concat([drug_all.iloc[:, :14], np_df],axis=1)
        
    drug_all.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # 查看数据
    # print(drug_all)
    # print(drug_all['Y'].value_counts())

    X = drug_all.iloc[:,6:]
    y = drug_all.iloc[:,5]

    results = nested_cv(X,y)

    # Extract and store the relevant information from the result

    params.append(results[0])
    aver_score.append(results[1])
    std_score.append(results[2])
    cm_score.append(results[3])
    accuracy_score.append(results[4])

    # Process and save the data
    data = {
        'network': cutoff,
        'params': params,
        'metrics_aver_score': aver_score,
        'metrics_std_score': std_score,
        'cm': cm_score,
        'accuracy':accuracy_score
    }
    result_all = pd.concat([result_all, pd.DataFrame(data)],axis=0,ignore_index=True)

timenow = time.strftime("%m%d%H%M_", time.localtime())
result_all.to_excel(result_path+timenow+algorithm+'.xlsx',index=False)
